### Part 1. Get the list of unlisted products

In [5]:
import pandas as pd 
import numpy as np
import selenium
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By

browser = webdriver.Chrome(r"C:\Program Files\Development\chromedriver.exe")
url = "https://apps.sfc.hk/productlistWeb/searchProduct/UTMF.do"
browser.get(url)

In [6]:
browser.find_element_by_xpath("//*[@id=\"searchResult_UTMFCard\"]/div[1]/div[2]/label[1]").click()
table = browser.find_elements_by_xpath("//*[@id=\"productListForm\"]/div[3]")[0]
fund_list = pd.read_html(table.get_attribute('outerHTML'), header=0)[0]


- 1: All

- 2: Funds under Chapter 7 of the Code on Unit Trusts and Mutual Funds (excluding funds with guaranteed features)

- 3: Unit Portfolio Management Funds (Ch 8.1)

- 4: Money Market Funds (Ch 8.2)

- 5: Futures and Options Funds (Ch 8.4A)

- 6: Funds with guaranteed features

- 7: Index Funds (Ch 8.6)

- 8: Hedge Funds (Ch 8.7)

- 9: Structured Funds (Ch 8.8)

- 10: Funds that invest extensively in financial derivative instruments (Ch 8.9)

- 11: UCITS Funds

- 12: Recognised Mainland Funds

- 13: Recognised Malaysian Islamic Funds

- 14: Recognised Luxembourg UCITS Funds

In [7]:
type_dict = {1: 'All', 2: 'Funds under Chapter 7 of the Code on Unit Trusts and Mutual Funds (excluding funds with guaranteed features)', 3: 'Unit Portfolio Management Funds (Ch 8.1)', 4: 'Money Market Funds (Ch 8.2)', 5: 'Futures and Options Funds (Ch 8.4A)', 6: 'Funds with guaranteed features', 7: 'Index Funds (Ch 8.6)', 8: 'Hedge Funds (Ch 8.7)', 9: 'Structured Funds (Ch 8.8)', 10: 'Funds that invest extensively in financial derivative instruments (Ch 8.9)', 11: 'UCITS Funds', 12: 'Recognised Mainland Funds', 13: 'Recognised Malaysian Islamic Funds', 14: 'Recognised Luxembourg UCITS Funds'}
for n in range(2,15):
#for n in range(2,3):
    browser.find_element_by_xpath("//*[@id=\"searchResult_UTMFCard\"]/div[1]/div[2]/label[{}]".format(n)).click()
    time.sleep(1)
    num_fund = int(re.findall(r'\d+', browser.find_element_by_xpath("//*[@id=\"productListForm\"]/p[1]").text)[0])
    if num_fund == 0:
        continue
    else:
        table = browser.find_elements_by_xpath("//*[@id=\"productListForm\"]/div[3]")[0]
        type_list = pd.read_html(table.get_attribute('outerHTML'), header=0)[0]
        type_list['type_{}'.format(n)] = n
        fund_list = pd.merge(fund_list, type_list.astype(str), how='left')
    time.sleep(1)

In [20]:
fund_list = pd.read_csv('data/temp/fund_list.csv')
type_list = fund_list.iloc[:,6:].fillna(0).astype(int).sum(axis=1)

fund_list = fund_list.iloc[:,0:6]
fund_list['type'] = type_list



In [21]:
fund_list.to_csv('data/temp/fund_list_type.csv', index=False)
